In [2]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.2.0
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: c:\users\hp\anaconda3\anaconda\lib\site-packages
Requires: joblib, scipy, threadpoolctl, numpy
Required-by: scikit-learn-intelex


In [104]:
import pandas as pd
import numpy as np

In [ ]:
car = pd.read_csv("https://raw.githubusercontent.com/rajtilakls2510/car_price_predictor/master/quikr_car.csv")
car.to_csv('Quickrr Dataset.csv')

In [41]:
car.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [42]:
car.shape

(892, 6)

In [43]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


In [45]:
# cleaning


backup = car.copy()

In [48]:
car=car[car['year'].str.isnumeric()]

In [49]:
car['year']=car['year'].astype(int)

In [51]:
car=car[car['Price']!="Ask For Price"]

In [55]:
car['Price']=car['Price'].str.replace(',','').astype(int)

In [59]:
car['kms_driven']=car['kms_driven'].str.split(' ').str.get(0).str.replace(',','')

In [61]:
car=car[car['kms_driven'].str.isnumeric()]

In [62]:
car['kms_driven']=car['kms_driven'].astype(int)

In [65]:
car=car[~car['fuel_type'].isna()]

In [69]:
car['name']=car['name'].str.split(' ').str.slice(0,3).str.join(' ')

In [71]:
car=car.reset_index(drop=True)

In [74]:
car=car[car['Price']<6e6].reset_index(drop=True)

In [76]:
car.to_csv('Cleaned Car.csv')

In [77]:
#model

x=car.drop(columns='Price')
y=car['Price']

In [79]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test=train_test_split(x,y,test_size=0.2)

In [84]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [83]:
ohe=OneHotEncoder()
ohe.fit(x[['name','company','fuel_type']])

OneHotEncoder()

In [90]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name','company','fuel_type']),remainder='passthrough')

In [91]:
lr=LinearRegression()

In [92]:
pipe=make_pipeline(column_trans,lr)

In [93]:
pipe.fit(X_train,Y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Pe

In [94]:
y_pred=pipe.predict(X_test)

In [100]:
r2_score(Y_test,y_pred)

0.7902990024738231

In [103]:
scores=[]
for i in range(1000):
    X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,Y_train)
    Y_pred=pipe.predict(X_test)
    scores.append(r2_score(Y_test,y_pred))

In [113]:
    X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2,random_state=661)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,Y_train)
    Y_pred=pipe.predict(X_test)
    r2_score(Y_test,Y_pred)

0.8897762920578806

In [115]:
import pickle

In [116]:
pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))

In [117]:
pipe.predict(pd.DataFrame([['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']],columns=['name','company','year','kms_driven','fuel_type']))

array([400821.27275562])